In [75]:
import csv
import math
import sys
from datetime import datetime as dt
from datetime import timezone as tz

import numpy as np
import pandas as pd

import utility

## clean data

### regional

In [198]:
df = pd.read_excel('../GhostPostCC/data/Region_MIDW.xlsx')

In [197]:
df

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Other Fuel Sources (Imputed),Net Generation (MW) from Coal (Adjusted),Net Generation (MW) from Natural Gas (Adjusted),Net Generation (MW) from Nuclear (Adjusted),Net Generation (MW) from All Petroleum Products (Adjusted),Net Generation (MW) from Hydropower and Pumped Storage (Adjusted),Net Generation (MW) from Solar (Adjusted),Net Generation (MW) from Wind (Adjusted),Net Generation (MW) from Other Fuel Sources (Adjusted),Region
0,AECI,01/01/2022,1,01/01/2022 1:00:00 AM,01/01/2022 7:00:00 AM,"2,235","2,251","1,986",-265,-265,...,NaN,996,299,NaN,NaN,NaN,NaN,691,NaN,MIDW
1,AECI,01/01/2022,2,01/01/2022 2:00:00 AM,01/01/2022 8:00:00 AM,"2,217","2,208","2,039",-169,-169,...,NaN,941,302,NaN,NaN,NaN,NaN,796,NaN,MIDW
2,AECI,01/01/2022,3,01/01/2022 3:00:00 AM,01/01/2022 9:00:00 AM,"2,193","2,204","2,080",-124,-124,...,NaN,951,369,NaN,NaN,NaN,NaN,760,NaN,MIDW
3,AECI,01/01/2022,4,01/01/2022 4:00:00 AM,01/01/2022 10:00:00 AM,"2,255","2,234","2,110",-124,-124,...,NaN,957,386,NaN,NaN,NaN,NaN,767,NaN,MIDW
4,AECI,01/01/2022,5,01/01/2022 5:00:00 AM,01/01/2022 11:00:00 AM,"2,325","2,287","2,138",-149,-149,...,NaN,992,405,NaN,NaN,NaN,NaN,741,NaN,MIDW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273616,YAD,06/30/2022,20,06/30/2022 8:00:00 PM,07/01/2022 12:00:00 AM,NaN,NaN,122,119,119,...,NaN,NaN,NaN,NaN,NaN,122.0,NaN,NaN,NaN,CAR
273617,YAD,06/30/2022,21,06/30/2022 9:00:00 PM,07/01/2022 1:00:00 AM,NaN,NaN,4,2,4,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,CAR
273618,YAD,06/30/2022,22,06/30/2022 10:00:00 PM,07/01/2022 2:00:00 AM,NaN,NaN,0,-1,-1,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,CAR
273619,YAD,06/30/2022,23,06/30/2022 11:00:00 PM,07/01/2022 3:00:00 AM,NaN,NaN,1,-1,-1,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,CAR


In [61]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77303 entries, 0 to 77302
Data columns (total 59 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   Region                                             77303 non-null  object        
 1   UTC time                                           77303 non-null  datetime64[ns]
 2   Local date                                         77303 non-null  datetime64[ns]
 3   Hour                                               77303 non-null  int64         
 4   Local time                                         77303 non-null  datetime64[ns]
 5   Time zone                                          77303 non-null  object        
 6   DF                                                 77303 non-null  int64         
 7   D                                                  77279 non-null  float64       
 8   NG              

In [62]:
df.columns

Index(['Region', 'UTC time', 'Local date', 'Hour', 'Local time', 'Time zone',
       'DF', 'D', 'NG', 'TI', 'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC',
       'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH', 'NG: UNK',
       'Sum (Trade)', 'Sum (Imports)', 'Sum (Exports)', 'CAL', 'CAR', 'CENT',
       'FLA', 'MIDA', 'MIDW', 'NE', 'NW', 'NY', 'SE', 'SW', 'TEN', 'TEX',
       'CAN', 'MEX', 'Balance NG D TI', 'Balance TI Trade', 'Balance NG',
       'CO2 Factor: COL', 'CO2 Factor: NG', 'CO2 Factor: OIL',
       'Positive Gen from COL', 'Positive Gen from NG',
       'Positive Gen from OIL', 'CO2 Emissions: COL', 'CO2 Emissions: NG',
       'CO2 Emissions: OIL', 'CO2 Emissions: Other', 'CO2 Emissions Generated',
       'CO2 Emissions Imported', 'CO2 Emissions Exported',
       'CO2 Emissions Consumed', 'Positive Generation', 'Consumed Electricity',
       'CO2 Emissions Intensity for Generated Electricity',
       'CO2 Emissions Intensity for Consumed Electricity'],
      dtype='obj

In [76]:
cols = ['UTC time', 'Local date', 'Hour', 'Local time',
       'Sum (NG)', 'NG: COL', 'NG: NG', 'NG: NUC', 'NG: SUN',
       'NG: OIL', 'NG: WAT', 'NG: SUN', 'NG: WND', 'NG: OTH']

time_filter = (df['Local time']>'2022-01-01') & (df['Local time']<='2024-01-01')

In [115]:
data = df[time_filter][cols].reset_index(drop=True)
data

,UTC time,Local date,Hour,Local time,Sum (NG),NG: COL,NG: NG,NG: NUC,NG: SUN,NG: OIL,NG: WAT,NG: SUN,NG: WND,NG: OTH
0,2022-01-01 06:00:00,2022-01-01,1,2022-01-01 01:00:00,64543.0,25663.0,12065.0,12192.0,0.0,NaN,569.0,0.0,13584.0,470.0
1,2022-01-01 07:00:00,2022-01-01,2,2022-01-01 02:00:00,63027.0,24282.0,11574.0,12192.0,0.0,NaN,529.0,0.0,13982.0,468.0
2,2022-01-01 08:00:00,2022-01-01,3,2022-01-01 03:00:00,62214.0,23480.0,11296.0,12192.0,0.0,NaN,522.0,0.0,14256.0,468.0
3,2022-01-01 09:00:00,2022-01-01,4,2022-01-01 04:00:00,61693.0,22871.0,11026.0,12198.0,0.0,NaN,529.0,0.0,14600.0,469.0
4,2022-01-01 10:00:00,2022-01-01,5,2022-01-01 05:00:00,61166.0,22423.0,10701.0,12195.0,0.0,NaN,527.0,0.0,14850.0,470.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2024-01-01 01:00:00,2023-12-31,20,2023-12-31 20:00:00,78078.0,29329.7,30487.3,8204.0,2.0,0.0,1088.0,2.0,8596.0,371.0
17516,2024-01-01 02:00:00,2023-12-31,21,2023-12-31 21:00:00,75593.9,27283.2,30073.8,8204.0,2.9,0.0,909.0,2.9,8753.0,368.0
17517,2024-01-01 03:00:00,2023-12-31,22,2023-12-31 22:00:00,74670.1,26415.1,30143.0,8206.0,3.0,0.0,905.0,3.0,8635.0,363.0
17518,2024-01-01 04:00:00,2023-12-31,23,2023-12-31 23:00:00,73296.0,25840.3,30098.7,8203.0,3.0,0.0,878.0,3.0,7908.0,365.0


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   UTC time    17520 non-null  datetime64[ns]
 1   Local date  17520 non-null  datetime64[ns]
 2   Hour        17520 non-null  int64         
 3   Local time  17520 non-null  datetime64[ns]
 4   Sum (NG)    17520 non-null  float64       
 5   NG: COL     17520 non-null  float64       
 6   NG: NG      17520 non-null  float64       
 7   NG: NUC     17520 non-null  float64       
 8   NG: SUN     17520 non-null  float64       
 9   NG: OIL     5184 non-null   float64       
 10  NG: WAT     17520 non-null  float64       
 11  NG: SUN     17520 non-null  float64       
 12  NG: WND     17520 non-null  float64       
 13  NG: OTH     17520 non-null  float64       
dtypes: datetime64[ns](3), float64(10), int64(1)
memory usage: 1.9 MB


In [117]:
# omit NG: SUN bcs no sun data
# omit NG: OIL bcs too many nans
# change NG: OTH to become (Sum (NG) - ('NG: COL' + 'NG: NG' + 'NG: NUC' + 'NG: WAT' + 'NG: SUN' + 'NG: WND'))

data = data.drop(['NG: SUN', 'NG: OIL'], axis=1)

data['NG: OTH'] = 0
data['NG: OTH'] = data['Sum (NG)'] - data[['NG: COL', 'NG: NG', 'NG: NUC', 'NG: WAT', 'NG: WND']].sum(axis=1)
data = data.drop(['Sum (NG)', 'Local date', 'Hour'], axis=1)

In [118]:
data

,UTC time,Local time,NG: COL,NG: NG,NG: NUC,NG: WAT,NG: WND,NG: OTH
0,2022-01-01 06:00:00,2022-01-01 01:00:00,25663.0,12065.0,12192.0,569.0,13584.0,470.0
1,2022-01-01 07:00:00,2022-01-01 02:00:00,24282.0,11574.0,12192.0,529.0,13982.0,468.0
2,2022-01-01 08:00:00,2022-01-01 03:00:00,23480.0,11296.0,12192.0,522.0,14256.0,468.0
3,2022-01-01 09:00:00,2022-01-01 04:00:00,22871.0,11026.0,12198.0,529.0,14600.0,469.0
4,2022-01-01 10:00:00,2022-01-01 05:00:00,22423.0,10701.0,12195.0,527.0,14850.0,470.0
...,...,...,...,...,...,...,...,...
17515,2024-01-01 01:00:00,2023-12-31 20:00:00,29329.7,30487.3,8204.0,1088.0,8596.0,373.0
17516,2024-01-01 02:00:00,2023-12-31 21:00:00,27283.2,30073.8,8204.0,909.0,8753.0,370.9
17517,2024-01-01 03:00:00,2023-12-31 22:00:00,26415.1,30143.0,8206.0,905.0,8635.0,366.0
17518,2024-01-01 04:00:00,2023-12-31 23:00:00,25840.3,30098.7,8203.0,878.0,7908.0,368.0


In [119]:
data.to_csv('data/electricity_cleaned.csv')

### six months

In [293]:
def preprocess_numeric_value(value):
    # If the value is a string containing a comma (thousand separator), remove the comma
    if isinstance(value, str) and ',' in value:
        return float(value.replace(',', ''))
    else:
        return value

def combine_data(balancing_auth):
    years = ['2021', '2022', '2023', '2024']
    periods = ['Jan_Jun', 'Jul_Dec']

    dataset = pd.DataFrame()
    for year in years:
        for period in periods:
            try:
                file_path = f"../GhostPostCC/data/EIA930_BALANCE_{year}_{period}.csv"
                df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
                df = df[df['Balancing Authority']==balancing_auth]

                # Preprocess and convert columns starting from the 5th index to float
                for col in df.columns[5:]:
                    try:
                        df[col] = df[col].apply(preprocess_numeric_value)
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                    except:
                        pass  # Skip columns that cannot be converted to numeric

                dataset = pd.concat([dataset, df]).reset_index(drop=True)
                print(f'{file_path} combined')
            except FileNotFoundError as e:
                print(e)

    return dataset

In [294]:
dataset = combine_data('MISO')

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: DtypeWarning: Columns (5,6,10,11,13,16,18,19,20,21,22,23,25,26,27,29,32,33,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2021_Jan_Jun.csv combined


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2021_Jul_Dec.csv combined


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


...

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: DtypeWarning: Columns (5,6,7,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2023_Jul_Dec.csv combined


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])
/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


../GhostPostCC/data/EIA930_BALANCE_2024_Jan_Jun.csv combined
[Errno 2] No such file or directory: '../GhostPostCC/data/EIA930_BALANCE_2024_Jul_Dec.csv'


/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/171228780.py:17: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(f'{file_path}', infer_datetime_format=True, parse_dates=['UTC Time at End of Hour', 'Local Time at End of Hour', 'Data Date'])


In [295]:
dataset

,Balancing Authority,Data Date,Hour Number,Local Time at End of Hour,UTC Time at End of Hour,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Sum(Valid DIBAs) (MW),...,Net Generation (MW) from Other Fuel Sources (Imputed),Net Generation (MW) from Coal (Adjusted),Net Generation (MW) from Natural Gas (Adjusted),Net Generation (MW) from Nuclear (Adjusted),Net Generation (MW) from All Petroleum Products (Adjusted),Net Generation (MW) from Hydropower and Pumped Storage (Adjusted),Net Generation (MW) from Solar (Adjusted),Net Generation (MW) from Wind (Adjusted),Net Generation (MW) from Other Fuel Sources (Adjusted),Region
0,MISO,2021-01-01,1,2021-01-01 01:00:00,2021-01-01 06:00:00,67059.0,67566.0,63940.0,-4521.0,-4521.0,...,NaN,25367.0,15999.0,11534.0,NaN,714.0,0.0,9486.0,839.0,NaN
1,MISO,2021-01-01,2,2021-01-01 02:00:00,2021-01-01 07:00:00,65069.0,66270.0,62553.0,-4435.0,-4435.0,...,NaN,24662.0,16063.0,11544.0,NaN,664.0,0.0,8780.0,840.0,NaN
2,MISO,2021-01-01,3,2021-01-01 03:00:00,2021-01-01 08:00:00,63767.0,65380.0,61389.0,-4673.0,-4673.0,...,NaN,25365.0,16057.0,11555.0,NaN,661.0,0.0,6912.0,839.0,NaN
3,MISO,2021-01-01,4,2021-01-01 04:00:00,2021-01-01 09:00:00,62861.0,64402.0,59712.0,-5251.0,-5251.0,...,NaN,24811.0,15870.0,11560.0,NaN,664.0,0.0,5967.0,841.0,NaN
4,MISO,2021-01-01,5,2021-01-01 05:00:00,2021-01-01 10:00:00,62396.0,63640.0,58947.0,-5408.0,-5408.0,...,NaN,25041.0,15755.0,11565.0,NaN,668.0,0.0,5074.0,844.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29035,MISO,2024-04-24,20,2024-04-24 20:00:00,2024-04-25 01:00:00,70461.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29036,MISO,2024-04-24,21,2024-04-24 21:00:00,2024-04-25 02:00:00,70246.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29037,MISO,2024-04-24,22,2024-04-24 22:00:00,2024-04-25 03:00:00,67846.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29038,MISO,2024-04-24,23,2024-04-24 23:00:00,2024-04-25 04:00:00,64743.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [297]:
cols_contain_to_omit = ['Hour Number', 'Data Date', 'Balancing Authority', 'Demand', 'Interchange', 'DIBAs', 'Imputed', 'Adjusted', 'Region']

def filter_columns(df, cols_contain_to_omit):
    for omit in cols_contain_to_omit:
        df = df.loc[:,~df.columns.str.contains(omit, case=False)]
    
    return df

In [298]:
dataset = filter_columns(dataset, cols_contain_to_omit)

In [299]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 12 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Local Time at End of Hour                               29040 non-null  datetime64[ns]
 1   UTC Time at End of Hour                                 29040 non-null  datetime64[ns]
 2   Net Generation (MW)                                     29011 non-null  float64       
 3   Net Generation (MW) from Coal                           29011 non-null  float64       
 4   Net Generation (MW) from Natural Gas                    29011 non-null  float64       
 5   Net Generation (MW) from Nuclear                        29011 non-null  float64       
 6   Net Generation (MW) from All Petroleum Products         24 non-null     float64       
 7   Net Generation (MW) from Hydropower and Pumped Storage  29

In [300]:
# all nans in the last 25 rows
dataset.tail(30)

,Local Time at End of Hour,UTC Time at End of Hour,Net Generation (MW),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources
29010,2024-04-23 19:00:00,2024-04-24 00:00:00,67906.0,12515.0,22389.0,9549.0,NaN,1809.0,1959.0,19403.0,284.0,NaN
29011,2024-04-23 20:00:00,2024-04-24 01:00:00,67450.0,13178.0,24567.0,9553.0,NaN,2320.0,251.0,17334.0,248.0,NaN
29012,2024-04-23 21:00:00,2024-04-24 02:00:00,66111.0,14894.0,25584.0,9557.0,NaN,1921.0,1.0,13903.0,252.0,NaN
29013,2024-04-23 22:00:00,2024-04-24 03:00:00,63665.0,14970.0,24634.0,9565.0,NaN,1799.0,0.0,12441.0,256.0,NaN
29014,2024-04-23 23:00:00,2024-04-24 04:00:00,60306.0,13664.0,23328.0,9567.0,NaN,1169.0,0.0,12319.0,259.0,NaN
29015,2024-04-24 00:00:00,2024-04-24 05:00:00,58468.0,12639.0,22016.0,9568.0,NaN,979.0,0.0,13005.0,260.0,NaN
29016,2024-04-24 01:00:00,2024-04-24 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29017,2024-04-24 02:00:00,2024-04-24 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29018,2024-04-24 03:00:00,2024-04-24 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29019,2024-04-24 04:00:00,2024-04-24 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [301]:
dataset = dataset[dataset['UTC Time at End of Hour']<'2024-01-01']

In [302]:
dataset.columns

Index(['Local Time at End of Hour', 'UTC Time at End of Hour',
       'Net Generation (MW)', 'Net Generation (MW) from Coal',
       'Net Generation (MW) from Natural Gas',
       'Net Generation (MW) from Nuclear',
       'Net Generation (MW) from All Petroleum Products',
       'Net Generation (MW) from Hydropower and Pumped Storage',
       'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind',
       'Net Generation (MW) from Other Fuel Sources',
       'Net Generation (MW) from Unknown Fuel Sources'],
      dtype='object')

In [306]:
dataset

,Local Time at End of Hour,UTC Time at End of Hour,Net Generation (MW),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Net Generation (MW) from Unknown Fuel Sources
0,2021-01-01 01:00:00,2021-01-01 06:00:00,63940.0,25367.0,15999.0,11534.0,NaN,714.0,0.0,9486.0,839.0,NaN
1,2021-01-01 02:00:00,2021-01-01 07:00:00,62553.0,24662.0,16063.0,11544.0,NaN,664.0,0.0,8780.0,840.0,NaN
2,2021-01-01 03:00:00,2021-01-01 08:00:00,61389.0,25365.0,16057.0,11555.0,NaN,661.0,0.0,6912.0,839.0,NaN
3,2021-01-01 04:00:00,2021-01-01 09:00:00,59712.0,24811.0,15870.0,11560.0,NaN,664.0,0.0,5967.0,841.0,NaN
4,2021-01-01 05:00:00,2021-01-01 10:00:00,58947.0,25041.0,15755.0,11565.0,NaN,668.0,0.0,5074.0,844.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
26269,2023-12-31 14:00:00,2023-12-31 19:00:00,68586.0,19663.0,26718.0,8205.0,NaN,811.0,1060.0,11799.0,331.0,NaN
26270,2023-12-31 15:00:00,2023-12-31 20:00:00,68042.0,19740.0,26912.0,8203.0,NaN,805.0,976.0,11074.0,333.0,NaN
26271,2023-12-31 16:00:00,2023-12-31 21:00:00,67827.0,20145.0,26794.0,8204.0,NaN,802.0,825.0,10717.0,340.0,NaN
26272,2023-12-31 17:00:00,2023-12-31 22:00:00,68717.0,21403.0,27195.0,8203.0,NaN,850.0,541.0,10181.0,343.0,NaN


In [307]:
dataset['Net Generation (MW) from Other'] = 0
dataset['Net Generation (MW) from Other'] = dataset['Net Generation (MW)'] - dataset[['Net Generation (MW) from Coal',
                                                 'Net Generation (MW) from Natural Gas',
                                                 'Net Generation (MW) from Nuclear',
                                                 'Net Generation (MW) from All Petroleum Products',
                                                 'Net Generation (MW) from Hydropower and Pumped Storage',
                                                 'Net Generation (MW) from Solar', 'Net Generation (MW) from Wind', 'Net Generation (MW) from Unknown Fuel Sources']].sum(axis=1)

In [312]:
dataset = dataset.drop(['Net Generation (MW) from Other Fuel Sources', 'Net Generation (MW) from Unknown Fuel Sources', 'Net Generation (MW)', 'Local Time at End of Hour'], axis=1)

In [313]:
dataset

,UTC Time at End of Hour,Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other
0,2021-01-01 06:00:00,25367.0,15999.0,11534.0,NaN,714.0,0.0,9486.0,840.0
1,2021-01-01 07:00:00,24662.0,16063.0,11544.0,NaN,664.0,0.0,8780.0,840.0
2,2021-01-01 08:00:00,25365.0,16057.0,11555.0,NaN,661.0,0.0,6912.0,839.0
3,2021-01-01 09:00:00,24811.0,15870.0,11560.0,NaN,664.0,0.0,5967.0,840.0
4,2021-01-01 10:00:00,25041.0,15755.0,11565.0,NaN,668.0,0.0,5074.0,844.0
...,...,...,...,...,...,...,...,...,...
26269,2023-12-31 19:00:00,19663.0,26718.0,8205.0,NaN,811.0,1060.0,11799.0,330.0
26270,2023-12-31 20:00:00,19740.0,26912.0,8203.0,NaN,805.0,976.0,11074.0,332.0
26271,2023-12-31 21:00:00,20145.0,26794.0,8204.0,NaN,802.0,825.0,10717.0,340.0
26272,2023-12-31 22:00:00,21403.0,27195.0,8203.0,NaN,850.0,541.0,10181.0,344.0


In [314]:
dataset.to_csv('data/MW_electricity_cleaned.csv')

## process data

In [349]:
import numpy as np
import pandas as pd
import pytz as pytz
from keras.layers import Dense, Flatten
from keras.models import Sequential
from scipy.sparse import data
from sklearn.utils import validation
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [315]:
source = 'coal'
IN_FILE_NAME = "data/MW_electricity_cleaned.csv"

In [331]:
NUM_FEATURES_DICT = {"coal":6, "nat_gas":6, "nuclear":6, "oil":6, "hydro":11, "solar": 11,
                    "wind":11, "unknown": 6, "biomass": 6, "geothermal":6}

NUM_VAL_DAYS = 30
NUM_TEST_DAYS = 184
TRAINING_WINDOW_HOURS = 24
PREDICTION_WINDOW_HOURS = 24
MODEL_SLIDING_WINDOW_LEN = 24

COAL = 1
NAT_GAS = 2
NUCLEAR = 3
OIL = 4
HYDRO = 5
SOLAR = 5
WIND = 6
OTHERS = 7 

FUEL = {COAL:"coal", NAT_GAS:"nat_gas", NUCLEAR:"nuclear", OIL:"oil", HYDRO:"hydro", SOLAR:"solar", WIND:"wind", OTHERS:"others"}
SOURCE_TO_SOURCE_COL_MAP = {y: x for x, y in FUEL.items()}

SOURCE_COL = SOURCE_TO_SOURCE_COL_MAP[source]
NUM_FEATURES = NUM_FEATURES_DICT[FUEL[SOURCE_COL]]

In [332]:
def initDataset(inFileName, sourceCol):
    dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC Time at End of Hour'], index_col=['UTC Time at End of Hour'])

    print(dataset.head())
    print(dataset.columns)
    dateTime = dataset.index.values
    
    print("\nAdding features related to date & time...")
    modifiedDataset = utility.addDateTimeFeatures(dataset, dateTime, sourceCol)
    dataset = modifiedDataset
    print("Features related to date & time added")
    
    for i in range(sourceCol, len(dataset.columns.values)):
        col = dataset.columns.values[i]
        dataset[col] = dataset[col].astype(np.float64)
        # print(col, dataset[col].dtype)

    return dataset, dateTime

In [333]:
dataset, dateTime = initDataset(IN_FILE_NAME, SOURCE_COL)

/var/folders/fh/7sz134vx5jq6_78w16smcs3h0000gn/T/ipykernel_54784/2017388480.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv(inFileName, header=0, infer_datetime_format=True, parse_dates=['UTC Time at End of Hour'], index_col=['UTC Time at End of Hour'])


                         Unnamed: 0  Net Generation (MW) from Coal  \
UTC Time at End of Hour                                              
2021-01-01 06:00:00               0                        25367.0   
2021-01-01 07:00:00               1                        24662.0   
2021-01-01 08:00:00               2                        25365.0   
2021-01-01 09:00:00               3                        24811.0   
2021-01-01 10:00:00               4                        25041.0   

                         Net Generation (MW) from Natural Gas  \
UTC Time at End of Hour                                         
2021-01-01 06:00:00                                   15999.0   
2021-01-01 07:00:00                                   16063.0   
2021-01-01 08:00:00                                   16057.0   
2021-01-01 09:00:00                                   15870.0   
2021-01-01 10:00:00                                   15755.0   

                         Net Generation (MW) from Nuc

In [334]:
dataset

,Unnamed: 0,Net Generation (MW) from Coal,hour_sin,hour_cos,month_sin,month_cos,weekend,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other
UTC Time at End of Hour,,,,,,,,,,,,,,
2021-01-01 06:00:00,0,25367.0,1.000000,6.123234e-17,0.008601,0.999963,0.0,15999.0,11534.0,NaN,714.0,0.0,9486.0,840.0
2021-01-01 07:00:00,1,24662.0,0.965926,-2.588190e-01,0.009318,0.999957,0.0,16063.0,11544.0,NaN,664.0,0.0,8780.0,840.0
2021-01-01 08:00:00,2,25365.0,0.866025,-5.000000e-01,0.010035,0.999950,0.0,16057.0,11555.0,NaN,661.0,0.0,6912.0,839.0
2021-01-01 09:00:00,3,24811.0,0.707107,-7.071068e-01,0.010751,0.999942,0.0,15870.0,11560.0,NaN,664.0,0.0,5967.0,840.0
2021-01-01 10:00:00,4,25041.0,0.500000,-8.660254e-01,0.011468,0.999934,0.0,15755.0,11565.0,NaN,668.0,0.0,5074.0,844.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,26269,19663.0,-0.965926,2.588190e-01,-0.012185,0.999926,1.0,26718.0,8205.0,NaN,811.0,1060.0,11799.0,330.0
2023-12-31 20:00:00,26270,19740.0,-0.866025,5.000000e-01,-0.011468,0.999934,1.0,26912.0,8203.0,NaN,805.0,976.0,11074.0,332.0
2023-12-31 21:00:00,26271,20145.0,-0.707107,7.071068e-01,-0.010751,0.999942,1.0,26794.0,8204.0,NaN,802.0,825.0,10717.0,340.0


In [340]:
trainData, valData, testData, fullTrainData = utility.splitDataset(dataset.values, NUM_TEST_DAYS, NUM_VAL_DAYS)
trainDates = dateTime[: -(NUM_TEST_DAYS*24)]
fullTrainDates = np.copy(trainDates)
trainDates, validationDates = trainDates[: -(NUM_VAL_DAYS*24)], trainDates[-(NUM_VAL_DAYS*24):]
testDates = dateTime[-(NUM_TEST_DAYS*24):]
trainData = trainData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]
valData = valData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]
testData = testData[:, SOURCE_COL: SOURCE_COL+NUM_FEATURES]

No. of rows in dataset: 26274
No. of rows in training set: 21138
No. of rows in validation set: 720
No. of rows in test set: 4416


In [341]:
pd.DataFrame(trainData)

,0,1,2,3,4,5
0,25367.0,1.000000,6.123234e-17,0.008601,0.999963,0.0
1,24662.0,0.965926,-2.588190e-01,0.009318,0.999957,0.0
2,25365.0,0.866025,-5.000000e-01,0.010035,0.999950,0.0
3,24811.0,0.707107,-7.071068e-01,0.010751,0.999942,0.0
4,25041.0,0.500000,-8.660254e-01,0.011468,0.999934,0.0
...,...,...,...,...,...,...
21133,23355.0,-0.965926,2.588190e-01,0.524316,-0.851524,0.0
21134,24151.0,-0.866025,5.000000e-01,0.523705,-0.851900,0.0
21135,23780.0,-0.707107,7.071068e-01,0.523094,-0.852275,0.0
21136,24451.0,-0.500000,8.660254e-01,0.522483,-0.852650,0.0


In [342]:
print("TrainData shape: ", trainData.shape) # (days x hour) x features
print("ValData shape: ", valData.shape) # (days x hour) x features
print("TestData shape: ", testData.shape) # (days x hour) x features
print("***** Dataset split done *****")

for i in range(trainData.shape[0]):
    for j in range(trainData.shape[1]):
        if(np.isnan(trainData[i, j])):
            trainData[i, j] = trainData[i-1, j]

for i in range(valData.shape[0]):
    for j in range(valData.shape[1]):
        if(np.isnan(valData[i, j])):
            valData[i, j] = valData[i-1, j]

for i in range(testData.shape[0]):
    for j in range(testData.shape[1]):
        if(np.isnan(testData[i, j])):
            testData[i, j] = testData[i-1, j]

featureList = dataset.columns.values[SOURCE_COL:SOURCE_COL+NUM_FEATURES]
print("Features: ", featureList)

TrainData shape:  (21138, 6)
ValData shape:  (720, 6)
TestData shape:  (4416, 6)
***** Dataset split done *****
Features:  ['Net Generation (MW) from Coal' 'hour_sin' 'hour_cos' 'month_sin'
 'month_cos' 'weekend']


In [344]:
print("Scaling data...")
# unscaledTestData = np.zeros(testData.shape[0])
# for i in range(testData.shape[0]):
#     unscaledTestData[i] = testData[i, 0]
trainData, valData, testData, ftMin, ftMax = utility.scaleDataset(trainData, valData, testData)
print("***** Data scaling done *****")
print(trainData.shape, valData.shape, testData.shape)

Scaling data...
***** Data scaling done *****
(21138, 6) (720, 6) (4416, 6)


In [345]:
# convert training data into inputs and outputs (labels)
def manipulateTrainingDataShape(data, trainWindowHours, labelWindowHours): 
    print("Data shape: ", data.shape)
    X, y = list(), list()
    # step over the entire history one time step at a time
    for i in range(len(data)-(trainWindowHours+labelWindowHours)+1):
        # define the end of the input sequence
        trainWindow = i + trainWindowHours
        labelWindow = trainWindow + labelWindowHours
        xInput = data[i:trainWindow, :]
        # xInput = xInput.reshape((len(xInput), 1))
        X.append(xInput)
        y.append(data[trainWindow:labelWindow, 0])
        # print(data[trainWindow:labelWindow, 0])
    return np.array(X, dtype=np.float64), np.array(y, dtype=np.float64)

def manipulateTestDataShape(data, slidingWindowLen, predictionWindowHours, isDates=False): 
    X = list()
    # step over the entire history one time step at a time
    for i in range(0, len(data)-(predictionWindowHours)+1, slidingWindowLen):
        # define the end of the input sequence
        predictionWindow = i + predictionWindowHours
        X.append(data[i:predictionWindow])
    if (isDates is False):
        X = np.array(X, dtype=np.float64)
    else:
        X = np.array(X)
    return X

In [346]:
print("\nManipulating training data...")
X, y = manipulateTrainingDataShape(trainData, TRAINING_WINDOW_HOURS, TRAINING_WINDOW_HOURS)
# Next line actually labels validation data
valX, valY = manipulateTrainingDataShape(valData, TRAINING_WINDOW_HOURS, TRAINING_WINDOW_HOURS)
print("***** Training data manipulation done *****")
print("X.shape, y.shape: ", X.shape, y.shape)


Manipulating training data...
Data shape:  (21138, 6)
Data shape:  (720, 6)
***** Training data manipulation done *****
X.shape, y.shape:  (21091, 24, 6) (21091, 24)


In [350]:
def trainANN(trainX, trainY, valX, valY, hyperParams):
    n_timesteps, n_features, nOutputs = trainX.shape[1], trainX.shape[2], trainY.shape[1]
    epochs = 1 #hyperParams['epoch']
    batchSize = hyperParams['batchsize']
    activationFunc = hyperParams['actv']
    lossFunc = hyperParams['loss']
    optimizer = hyperParams['optim']
    hiddenDims = hyperParams['hidden']
    learningRates = hyperParams['lr']
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(hiddenDims[0], input_shape=(n_timesteps, n_features), activation=activationFunc)) # 20 for coal, nat_gas, nuclear
    model.add(Dense(hiddenDims[1], activation='relu')) # 50 for coal, nat_gas, nuclear
    model.add(Dense(nOutputs))

    opt = tf.keras.optimizers.Adam(learning_rate = learningRates)
    model.compile(loss=lossFunc, optimizer=optimizer[0],
                    metrics=['mean_absolute_error'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    mc = ModelCheckpoint('best_model_ann.keras', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    # fit network
    hist = model.fit(trainX, trainY, epochs=epochs, batch_size=batchSize[0], verbose=2,
                        validation_data=(valX, valY), callbacks=[es, mc])
    model = load_model("best_model_ann.keras")
    utility.showModelSummary(hist, model)
    return model, n_features

def getDayAheadForecasts(trainX, trainY, model, history, testData, 
                            trainWindowHours, numFeatures, depVarColumn):
    global MODEL_SLIDING_WINDOW_LEN
    global PREDICTION_WINDOW_HOURS
    # walk-forward validation over each day
    print("Testing...")
    predictions = list()
    for i in range(0, len(testData)//24):
        dayAheadPredictions = list()
        tempHistory = history.copy()
        currentDayHours = i* MODEL_SLIDING_WINDOW_LEN
        for j in range(0, PREDICTION_WINDOW_HOURS, 24):
            yhat_sequence, newTrainingData = getForecasts(model, tempHistory, trainWindowHours, numFeatures)
            dayAheadPredictions.extend(yhat_sequence)
            # add current prediction to history for predicting the next day
            # following 3 lines are redundant currently. Will become useful if
            # prediction period goes beyond 24 hours.
            latestHistory = testData[currentDayHours+j:currentDayHours+j+24, :].tolist()
            for k in range(24):
                latestHistory[k][depVarColumn] = yhat_sequence[k]
            tempHistory.extend(latestHistory)

        # get real observation and add to history for predicting the next day
        history.extend(testData[currentDayHours:currentDayHours+MODEL_SLIDING_WINDOW_LEN, :].tolist())
        predictions.append(dayAheadPredictions)

    # evaluate predictions days for each day
    predictedData = np.array(predictions, dtype=np.float64)
    return predictedData


def getForecasts(model, history, trainWindowHours, numFeatures):
    # flatten data
    data = np.array(history, dtype=np.float64)
    # retrieve last observations for input data
    input_x = data[-trainWindowHours:]
    # reshape into [1, n_input, num_features]
    input_x = input_x.reshape((1, len(input_x), numFeatures))
    # print("ip_x shape: ", input_x.shape)
    yhat = model.predict(input_x, verbose=0)
    # we only want the vector forecast
    yhat = yhat[0]
    return yhat, input_x

def getANNHyperParams():
    hyperParams = {}
    hyperParams['epoch'] = 100 
    hyperParams['batchsize'] = [10] 
    hyperParams['actv'] = "relu"
    hyperParams['loss'] = "mse"
    hyperParams['optim'] = ["adam"] #, "rmsprop"]
    hyperParams['lr'] = 1e-2 #, 1e-3
    hyperParams['hidden'] = [20, 50] #, [50, 50]]#, [20, 50]] #, [50, 50]]
    return hyperParams

In [363]:
pd.DataFrame(valData)

,0,1,2,3,4,5
0,0.484295,0.500000,1.000000,0.760630,0.073301,0.0
1,0.490293,0.629410,0.982963,0.760324,0.073114,0.0
2,0.481907,0.750000,0.933013,0.760018,0.072928,0.0
3,0.432190,0.853553,0.853553,0.759712,0.072742,0.0
4,0.370022,0.933013,0.750000,0.759406,0.072556,0.0
...,...,...,...,...,...,...
715,0.604878,0.017037,0.629410,0.517915,0.000321,0.0
716,0.611103,0.066987,0.750000,0.517557,0.000308,0.0
717,0.622388,0.146447,0.853553,0.517199,0.000296,0.0
718,0.626652,0.250000,0.933013,0.516841,0.000284,0.0


In [368]:
pd.DataFrame(valData[-TRAINING_WINDOW_HOURS:, :])

,0,1,2,3,4,5
0,0.532022,0.500000,1.000000,0.524718,0.000611,0.0
1,0.528327,0.629410,0.982963,0.524360,0.000594,0.0
2,0.538190,0.750000,0.933013,0.524002,0.000576,0.0
3,0.525740,0.853553,0.853553,0.523645,0.000559,0.0
4,0.531567,0.933013,0.750000,0.523287,0.000543,0.0
5,0.507377,0.982963,0.629410,0.522929,0.000526,0.0
6,0.462833,1.000000,0.500000,0.522571,0.000510,0.0
7,0.404474,0.982963,0.370590,0.522212,0.000494,0.0
8,0.350749,0.933013,0.250000,0.521854,0.000478,0.0
9,0.316211,0.853553,0.146447,0.521496,0.000462,0.0


In [361]:
testData[:, 0]

array([0.6257142 , 0.63293442, 0.60621393, ..., 0.34134   , 0.37709997,
       0.41928423])

In [352]:
######################## START #####################

hyperParams = getANNHyperParams()
bestRMSE = []

OUT_FILE_NAME_PREFIX = 'data/src_prod_forecast'

OUT_FILE_NAME = OUT_FILE_NAME_PREFIX + "_" + featureList[0] + ".csv"
print("\nStarting training")
bestModel, numFeatures = trainANN(X, y, valX, valY, hyperParams)
print("***** Training done *****")
history = valData[-TRAINING_WINDOW_HOURS:, :].tolist()
predictedData = getDayAheadForecasts(X, y, bestModel, history, testData, 
                TRAINING_WINDOW_HOURS, numFeatures, 0)            
actualData = manipulateTestDataShape(testData[:, 0], 
        MODEL_SLIDING_WINDOW_LEN, PREDICTION_WINDOW_HOURS, False)
formattedTestDates = manipulateTestDataShape(testDates, 
        MODEL_SLIDING_WINDOW_LEN, PREDICTION_WINDOW_HOURS, True)
formattedTestDates = np.reshape(formattedTestDates, 
        formattedTestDates.shape[0]*formattedTestDates.shape[1])
actualData = actualData.astype(np.float64)
print("ActualData shape: ", actualData.shape)
actual = np.reshape(actualData, actualData.shape[0]*actualData.shape[1])
print("actual.shape: ", actual.shape)
unscaledTestData = utility.inverseDataScaling(actual, ftMax[0], 
                ftMin[0])
predictedData = predictedData.astype(np.float64)
print("PredictedData shape: ", predictedData.shape)
predicted = np.reshape(predictedData, predictedData.shape[0]*predictedData.shape[1])
print("predicted.shape: ", predicted.shape)
unScaledPredictedData = utility.inverseDataScaling(predicted, 
        ftMax[0], ftMin[0])
rmseScore, mapeScore = utility.getScores(actualData, predictedData, 
                        unscaledTestData, unScaledPredictedData)
print("***** Forecast done *****")
print("Overall RMSE score: ", rmseScore)
bestRMSE.append(rmseScore)


Starting training (iteration  0 )...

Epoch 1: val_loss improved from inf to 0.00501, saving model to best_model_ann.keras
2110/2110 - 2s - 830us/step - loss: 0.0085 - mean_absolute_error: 0.0631 - val_loss: 0.0050 - val_mean_absolute_error: 0.0571
Showing model summary...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 144)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │         2,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         1,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 24)             │         1,224 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,524 (60.64 KB)

 Trainable params: 5,174 (20.21 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,350 (40.43 KB)

***** Model summary shown *****
dict_keys(['loss', 'mean_absolute_error', 'val_loss', 'val_mean_absolute_error'])
***** Training done *****
Testing...
ActualData shape:  (184, 24)
actual.shape:  (4416,)
PredictedData shape:  (184, 24)
predicted.shape:  (4416,)
Actual data shape, Predicted data shape:  (184, 24) (184, 24)
***** Forecast done *****
Overall RMSE score:  0.077688


In [ ]:
data = []
for i in range(len(unScaledPredictedData)):
    row = []
    row.append(str(formattedTestDates[i]))
    row.append(str(unscaledTestData[i]))
    row.append(str(unScaledPredictedData[i]))
    data.append(row)
utility.writeOutFuelForecastFile(OUT_FILE_NAME, data, featureList[0])

    
print("Average RMSE after ", 1, " expts: ", np.mean(bestRMSE))
print(bestRMSE)

In [354]:
data = []
for i in range(len(unScaledPredictedData)):
    row = []
    row.append(str(formattedTestDates[i]))
    row.append(str(unscaledTestData[i]))
    row.append(str(unScaledPredictedData[i]))
    data.append(row)

In [355]:
data

[['2023-07-01T00:00:00.000000000', '30149.0', '29177.53916'],
 ['2023-07-01T01:00:00.000000000', '30403.0', '29624.76403'],
 ['2023-07-01T02:00:00.000000000', '29463.0', '29434.62961'],
 ['2023-07-01T03:00:00.000000000', '29036.0', '28514.80402'],
 ['2023-07-01T04:00:00.000000000', '28262.0', '27552.96211'],
 ['2023-07-01T05:00:00.000000000', '27518.0', '26243.23484'],
 ['2023-07-01T06:00:00.000000000', '26148.0', '25002.14107'],
 ['2023-07-01T07:00:00.000000000', '23665.0', '23342.58048'],
 ['2023-07-01T08:00:00.000000000', '21098.0', '22492.31518'],
 ['2023-07-01T09:00:00.000000000', '19682.0', '21887.07307'],
 ['2023-07-01T10:00:00.000000000', '19151.0', '21452.50678'],
 ['2023-07-01T11:00:00.000000000', '19319.0', '21718.89668'],
 ['2023-07-01T12:00:00.000000000', '18850.0', '22767.1962'],
 ['2023-07-01T13:00:00.000000000', '19039.0', '23870.7787'],
 ['2023-07-01T14:00:00.000000000', '21259.0', '25645.47003'],
 ['2023-07-01T15:00:00.000000000', '24176.0', '26411.61882'],
 ['2023-07